#### Parameters
- The set of engines that needs maintenance is $M = {1,2,...j}$
- The planning horizon is $T = {1,2,...,t}$
- The available groups are $G = {1,2,..g}$
- The number of working days of  team g on engine j is $\mu_{g,j}$
- Let $P_{g,j,t}$ be the penalty if team $g$ starts working on engine $j$ from day $t$
- Let $Q_j$ be the cost of engine $j$ if the maintenance is not done during the planning horizon $T$ 

#### Objective function 
$min$  $Z = \sum_{j=1}^{M} \sum_{g=1}^{G} \sum_{t=1}^{T} P_{g,j,t}* X_{g,j,t} + \sum_{j=1}^{M} (( 1 - \sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}) * Q_j)$

#### Decision Variable
\begin{equation}
  X_{g,j,t} = \left \{
  \begin{aligned}
    &1, && \text{if team g starts working on engine j at day t}\  \\
    &0, && \text{otherwise}
  \end{aligned} \right.
\end{equation} 

#### Constraints 
##### Maintenance of engine j can be performed at most once during the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t}\leq 1$ for all $j \in M$

##### Maintenance must be completed within the planning period
$\sum_{g=1}^{G} \sum_{t=1}^{T} X_{g,j,t} * (t + \mu_{g,j} -1) \leq T$ for all $j \in M$

##### Teams can only start on one engine a day
$\sum_{j=1}^{M} X_{g,j,t} \leq 1$ for all $t \in T, g \in G$

##### Teams can only work on one engine at a time
$K * X_{g,j,t_a} + (\sum_{t=t_a+1}^{min \{ T,t + \mu_{g,j} -1\}} \sum_{j=1}^{M} X_{g,j,t}) \leq K$ for all $g \in G, t_a \in T, j \in M$ where $K>1$



### 1. Linear model

In [1]:
# Global imports
import pandas as pd
from pulp import *
from pulp import solvers as pulp_solver_classes

In [2]:
df = pd.read_csv('../data/RUL_consultancy_predictions_A3.csv', sep=';', index_col='id')

In [3]:
M = list(df[df['RUL'] <= 25].index)
#M=list(df.index)

G = {1: 'A',
     2: 'A',
     3: 'B',
     4: 'B'}

c = {i:5 for i in range(1,21)}
c.update({i:7 for i in range(21,41)})
c.update({i:9 for i in range(41,61)})
c.update({i:5 for i in range(61,81)})
c.update({i:3 for i in range(81,101)})

mu_a = {i:4 for i in range(1,26)}
mu_a.update({i:6 for i in range(26,51)})
mu_a.update({i:3 for i in range(51,76)})
mu_a.update({i:5 for i in range(76,101)})
mu_b = {i:mu_a[i]+1 for i in range(1,34)}
mu_b.update({i:mu_a[i]+2 for i in range(34,68)})
mu_b.update({i:mu_a[i]+1 for i in range(68,101)})

mu = {g: {j: mu_a[j] if typ=='A' else mu_b[j] for j in M} for g, typ in G.items()}

T = 25

In [4]:
def get_penalty(g, j, t):
    '''
    Calculates penalty if team g starts working on engine j at day t
    '''
    RUL = df.loc[j].values[0]
    c_per_day = c[j]
    nr_days_costs = t - RUL + mu[g][j] - 1 # CHECK IF THIS IS CORRECT OR SHOULD BE - (RUL + 1)

    if nr_days_costs > 0:
        costs = nr_days_costs * c_per_day
        return costs
    else:
        return 0
    
def get_unmaintained_penalty(j, T):
    '''
    Calculates cost of engine j if the maintenance is not done during the planning horizon T 
    '''
    RUL = df.loc[j].values[0]
    c_per_day = c[j]
    nr_days_costs = T - RUL # CHECK IF THIS IS CORRECT OR SHOULD BE - (RUL + 1)
    
    if nr_days_costs > 0:
        costs = nr_days_costs * c_per_day
        return costs
    else: return 0

In [5]:
P = {g:{j:{t: get_penalty(g,j,t) for t in range(1, T+1)} for j in M} for g in G}
Q = {j: get_unmaintained_penalty(j, T) for j in M}

### Pulp Model

In [6]:
# Create the model
model = LpProblem(name='maintenance-schedule-optimization-1', sense=LpMinimize)

team_engine_day = [(g, j, t) for g in G for j in M for t in range(1, T+1)]

# Set LpVariable parameters
# Natural constraints included
X = LpVariable.dicts(name='start_day', indexs=team_engine_day, cat='Binary')

# Add objective function
model += lpSum(P[g][j][t] * X[(g, j, t)] for g in G for j in M for t in range(1, T+1)) + \
 lpSum((1 - (lpSum(X[(g, j, t)] for g in G for t in range(1, T+1)))) * Q[j] for j in M)

# Add constraints
# Maintenance for each engine at most once during planning horizon
for j in M:
    model += lpSum(X[(g, j, t)] for g in G for t in range(1, T+1)) <= 1

# Maintenance must be completed within the planning period
for j in M:
    model += lpSum(X[(g, j, t)] * (t + mu[g][j] - 1) for g in G for t in range(1, T+1)) <= T
    
# # # Teams can only start on one engine a day
for g in G:
     for t in range(1, T+1):
            model += lpSum(X[(g, j, t)] for j in M) <= 1

#Teams can only work at one engine at a time
K = 100
for g in G:
    for t_a in range(1, T+1):
        for j in M:
            model += K * X[(g, j, t_a)] + lpSum(X[(g, j, t)] \
                                                for t in range(t_a+1, min(t_a + mu[g][j],T)) for j in M) <= K


In [7]:
model.writeLP("maintenance.lp")

[start_day_(1,_100,_1),
 start_day_(1,_100,_10),
 start_day_(1,_100,_11),
 start_day_(1,_100,_12),
 start_day_(1,_100,_13),
 start_day_(1,_100,_14),
 start_day_(1,_100,_15),
 start_day_(1,_100,_16),
 start_day_(1,_100,_17),
 start_day_(1,_100,_18),
 start_day_(1,_100,_19),
 start_day_(1,_100,_2),
 start_day_(1,_100,_20),
 start_day_(1,_100,_21),
 start_day_(1,_100,_22),
 start_day_(1,_100,_23),
 start_day_(1,_100,_24),
 start_day_(1,_100,_25),
 start_day_(1,_100,_3),
 start_day_(1,_100,_4),
 start_day_(1,_100,_5),
 start_day_(1,_100,_6),
 start_day_(1,_100,_7),
 start_day_(1,_100,_8),
 start_day_(1,_100,_9),
 start_day_(1,_20,_1),
 start_day_(1,_20,_10),
 start_day_(1,_20,_11),
 start_day_(1,_20,_12),
 start_day_(1,_20,_13),
 start_day_(1,_20,_14),
 start_day_(1,_20,_15),
 start_day_(1,_20,_16),
 start_day_(1,_20,_17),
 start_day_(1,_20,_18),
 start_day_(1,_20,_19),
 start_day_(1,_20,_2),
 start_day_(1,_20,_20),
 start_day_(1,_20,_21),
 start_day_(1,_20,_22),
 start_day_(1,_20,_23),
 s

In [8]:
def results_lp(model):
    engine = []
    #Print the status of solving
    print("Status = %s" % LpStatus[model.status])
    # Print the value of the objective
    print("Objective = %f" % value(model.objective))

    #Print the value of the variables when value > 0 
    for v in model.variables():
        if v.varValue > 0:
            print(v.name, "=", v.varValue)
            engine.append(int(v.name.split("_")[3][:-1]))
    return engine

In [9]:
model.solve(pulp_solver_classes.PULP_CBC_CMD(maxSeconds=3600))
results_lp(model)

Status = Optimal
Objective = 65.000000
start_day_(1,_37,_15) = 1.0
start_day_(1,_49,_9) = 1.0
start_day_(1,_61,_21) = 1.0
start_day_(1,_68,_6) = 1.0
start_day_(1,_76,_1) = 1.0
start_day_(2,_35,_1) = 1.0
start_day_(2,_42,_7) = 1.0
start_day_(2,_56,_14) = 1.0
start_day_(2,_64,_22) = 1.0
start_day_(2,_82,_17) = 1.0
start_day_(3,_20,_9) = 1.0
start_day_(3,_34,_1) = 1.0
start_day_(3,_41,_14) = 1.0
start_day_(4,_31,_7) = 1.0
start_day_(4,_66,_14) = 1.0
start_day_(4,_81,_1) = 1.0
start_day_(4,_92,_19) = 1.0


[37, 49, 61, 68, 76, 35, 42, 56, 64, 82, 20, 34, 41, 31, 66, 81, 92]

In [10]:
model.solve(pulp_solver_classes.PULP_CBC_CMD(maxSeconds=60*60*4))
results_lp(model)

Status = Optimal
Objective = 79.000000
start_day_(1,_41,_18) = 1.0
start_day_(1,_49,_9) = 1.0
start_day_(1,_66,_15) = 1.0
start_day_(1,_68,_6) = 1.0
start_day_(1,_76,_1) = 1.0
start_day_(2,_34,_3) = 1.0
start_day_(2,_37,_16) = 1.0
start_day_(2,_42,_9) = 1.0
start_day_(2,_64,_22) = 1.0
start_day_(3,_20,_10) = 1.0
start_day_(3,_35,_1) = 1.0
start_day_(3,_82,_15) = 1.0
start_day_(4,_31,_7) = 1.0
start_day_(4,_56,_14) = 1.0
start_day_(4,_61,_19) = 1.0
start_day_(4,_81,_1) = 1.0


[41, 49, 66, 68, 76, 34, 37, 42, 64, 20, 35, 82, 31, 56, 61, 81]

In [11]:
model.solve(pulp_solver_classes.PULP_CBC_CMD(maxSeconds=60*60*8))
results_lp(model)

Status = Optimal
Objective = 44.000000
start_day_(1,_34,_1) = 1.0
start_day_(1,_36,_19) = 1.0
start_day_(1,_42,_7) = 1.0
start_day_(1,_56,_16) = 1.0
start_day_(1,_68,_13) = 1.0
start_day_(2,_20,_6) = 1.0
start_day_(2,_37,_16) = 1.0
start_day_(2,_49,_10) = 1.0
start_day_(2,_64,_22) = 1.0
start_day_(2,_76,_1) = 1.0
start_day_(3,_31,_7) = 1.0
start_day_(3,_61,_19) = 1.0
start_day_(3,_66,_14) = 1.0
start_day_(3,_81,_1) = 1.0
start_day_(4,_35,_1) = 1.0
start_day_(4,_41,_15) = 1.0
start_day_(4,_82,_9) = 1.0


[34, 36, 42, 56, 68, 20, 37, 49, 64, 76, 31, 61, 66, 81, 35, 41, 82]